In [44]:
import jsondiff as jd
import json
from deepdiff import DeepDiff
import copy
import oracle
import io

In [12]:
learn1 = json.loads(open("resources.json").read())
learn2 = json.loads(open("resources-double.json").read())
test = json.loads(open("resources_test.json").read())

In [117]:
ddiff = DeepDiff(learn1, learn2, ignore_order=False, view='tree')
ddiff.keys()

dict_keys(['dictionary_item_added', 'dictionary_item_removed', 'values_changed'])

In [111]:
def look_for_resources_diff(learn, test):
    f = io.StringIO()
    learn_trim = copy.deepcopy(learn)
    test_trim = copy.deepcopy(test)
    diff = jd.diff(learn_trim, test_trim, syntax="symmetric")

    print("[RESOURCE DIFF REPORT]", file=f)
    for rtype in diff:
        rdiff = diff[rtype]
        # Check for resource level delete/insert
        if jd.delete in rdiff:
            # Any resource deleted
            for (idx, item) in rdiff[jd.delete]:
                print(
                    "[resource deleted]",
                    rtype,
                    item["metadata"]["namespace"],
                    item["metadata"]["name"],
                    file=f,
                )
        if jd.insert in rdiff:
            # Any resource added
            for (idx, item) in rdiff[jd.insert]:
                print(
                    "[resource added]",
                    rtype,
                    item["metadata"]["namespace"],
                    item["metadata"]["name"],
                    file=f,
                )

        # Check for resource internal fields
        for idx in rdiff:
            if not type(idx) is int:
                continue
            resource = test[rtype][idx]
            name = resource["metadata"]["name"]
            namespace = resource["metadata"]["namespace"]
            item = rdiff[idx]
            for majorfield in item:
                if majorfield == jd.delete:
                    print("[field deleted]", item[majorfield], file=f)
                    continue
                if majorfield == jd.insert:
                    print("[field added]", item[majorfield], file=f)
                    continue
                data = item[majorfield]
                for subfield in data:
                    if True:
                        if type(data[subfield]) is list:
                            if 'SIEVE-IGNORE' in data[subfield][0]:
                                continue
                        if 'SIEVE-IGNORE' in str(data[subfield]):
                            continue
                        print(
                            "[%s field changed]" % (majorfield),
                            rtype,
                            name,
                            subfield,
                            "changed",
                            "delta: ",
                            data[subfield],
                            file=f,
                        )
                if jd.delete in data:
                    print(
                        "[%s field deleted]" % (majorfield),
                        rtype,
                        name,
                        data[jd.delete],
                        file=f,
                    )
                if jd.insert in data:
                    print(
                        "[%s field added]" % (majorfield),
                        rtype,
                        name,
                        data[jd.insert],
                        file=f,
                    )

    result = f.getvalue()
    f.close()
    return result

In [124]:
def nested_set(dic, keys, value):
    for key in keys[:-1]:
        dic = dic[key]
    dic[keys[-1]] = value

stored_learn = copy.deepcopy(learn2)
for key in ddiff['values_changed']:
#     print(key.path(output_format='list'))
    nested_set(stored_learn, key.path(output_format='list'), "SIEVE-IGNORE")
    
for key in ddiff['dictionary_item_added']:
#     print(key.path(output_format='list'))
    nested_set(stored_learn, key.path(output_format='list'), "SIEVE-IGNORE")
    
# stored_learn
print(look_for_resources_diff(stored_learn, test))



[RESOURCE DIFF REPORT]
[metadata field changed] deployment nifikop-operator annotations changed delta:  {delete: {'meta.helm.sh/release-name': 'nifikop-operator', 'meta.helm.sh/release-namespace': 'default'}}
[spec field changed] deployment nifikop-operator template changed delta:  {'spec': {'containers': {0: {'image': ['laphets/nifikop-operator:learn', 'laphets/nifikop-operator:atomicity-violation']}}}}
[metadata field changed] pod nifikop-operator-6b7697f96-pssvk generateName changed delta:  ['nifikop-operator-6756f8bf87-', 'nifikop-operator-6b7697f96-']
[metadata field changed] pod nifikop-operator-6b7697f96-pssvk labels changed delta:  {'pod-template-hash': ['6756f8bf87', '6b7697f96']}
[spec field changed] pod nifikop-operator-6b7697f96-pssvk nodeName changed delta:  ['kind-worker', 'kind-worker2']
[status field changed] pod nifikop-operator-6b7697f96-pssvk hostIP changed delta:  ['172.18.0.2', '172.18.0.3']
[status field changed] pod simplenifi-1-nodeq5jbw hostIP changed delta:  [

In [165]:
learn = json.loads(open("resources_rl.json").read())
test = json.loads(open("resources_rt.json").read())

import deepdiff

def nested_get(dic, keys):    
    for key in keys:
        dic = dic[key]
    return dic

tdiff = DeepDiff(learn, test, ignore_order=False, view='tree')
stored_test = copy.deepcopy(test)
not_care_keys = set(['annotations', 'managedFields', 'image', 'imageID', 'nodeName', 'hostIP', 'message', 'labels', 'generateName', 'ownerReferences'])

# not_care_keys = set()

for t in tdiff:
    for key in tdiff[t]:
#         print(key)
        path = key.path(output_format='list')
        if key.t1 != "SIEVE-IGNORE":
            has_not_care = False
            for kp in path:
                if kp in not_care_keys:
                    has_not_care = True
                    break
            if has_not_care:
                continue
            rType = path[0]
            print(key.t2)
            if str(key.t2) == 'not present':
                print("12233232", type(key.t2))
            
            if len(path) == 2 and type(key.t2) is deepdiff.helper.NotPresent:
                source = learn
            else:
                source = test

            print(t, path, len(test['persistentvolumeclaim']))
            print(key.t1)
            name = nested_get(source, path[:2] + ['metadata', 'name'])
            namespace = nested_get(source, path[:2] + ['metadata', 'namespace'])
            print(t, rType, namespace, name, path, key.t1, " => ", key.t2)


not present
12233232 <class 'deepdiff.helper.NotPresent'>
iterable_item_removed ['persistentvolumeclaim', 2] 2
{'metadata': {'creationTimestamp': 'SIEVE-IGNORE', 'finalizers': ['kubernetes.io/pvc-protection'], 'labels': {'app.kubernetes.io/component': 'rabbitmq', 'app.kubernetes.io/name': 'rabbitmq-cluster', 'app.kubernetes.io/part-of': 'rabbitmq'}, 'managedFields': [{'apiVersion': 'v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:labels': {'.': {}, 'f:app.kubernetes.io/component': {}, 'f:app.kubernetes.io/name': {}, 'f:app.kubernetes.io/part-of': {}}, 'f:ownerReferences': {'.': {}, 'k:{"uid":"3366d479-0665-4ae8-a773-24536e20fa9b"}': 'SIEVE-IGNORE'}}, 'f:spec': {'f:accessModes': {}, 'f:resources': {'f:requests': {'.': {}, 'f:storage': {}}}, 'f:volumeMode': {}}, 'f:status': {'f:phase': {}}}, 'manager': 'kube-controller-manager', 'operation': 'Update', 'time': 'SIEVE-IGNORE'}], 'name': 'persistence-rabbitmq-cluster-server-2', 'namespace': 'default', 'ownerReferences': [{'api